## Table 3: CA and TX White Applications Panel DiD Estimates 

from White Flight in Higher Education? The Case of Hispanic-Serving Institutions

by Laura Hamilton, Charlie Eaton, and Simon Cheng

In [1]:
qui {
   * use data/hsi_applications, clear 
    use data/hsi_ca_tx_applications, clear

   * use data/hsi_white_enrollment_final2, clear
    drop if lnWhite_app==. | lnpopulationwh==. 
   
   drop hispserveyear
    gen hispserveyear=year if hispserve==1
   bysort unitid: egen hispserveyearall=min(hispserveyear)
gen treated=0
replace treated=1 if year>=hispserveyearall
    drop if year>2019 | year<2000
    bysort unitid: egen maxyear=max(year)
bysort unitid: egen minyear=min(year)
    gen missingpanel=.
    bysort unitid (year): replace missingpanel=1 if year[_n-1]!=year-1
    replace missingpanel=. if year==2000
    bysort unitid: egen missingpanelall=min(missingpanel)
    replace missingpanelall=1 if maxyear!=2019 | minyear !=2000
    
    bysort unitid: egen alwaystreated=min(treated)
bysort unitid: egen evertreated=max(treated)
}

In [2]:
egen gvar=csgvar(treated), tvar(year) ivar(unitid)
drop if alwaystreated==1
drop if missingpanelall==1
rename treated SimpleATT

(375 observations deleted)
(67 observations deleted)


In [3]:
eststo: reghdfe lnWhite_app lnpop* lncpop* SimpleATT, absorb(unitid year) cluster(unitid)

(MWFE estimator converged in 2 iterations)

HDFE Linear regression                            Number of obs   =        840
Absorbing 2 HDFE groups                           F(  11,     41) =       6.02
Statistics robust to heteroskedasticity           Prob > F        =     0.0000
                                                  R-squared       =     0.9732
                                                  Adj R-squared   =     0.9707
                                                  Within R-sq.    =     0.1825
Number of clusters (unitid)  =         42         Root MSE        =     0.2237

                                (Std. err. adjusted for 42 clusters in unitid)
------------------------------------------------------------------------------
             |               Robust
 lnWhite_app | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
lnpopulati~m |   3.691883   1.369466     2.70   0.

In [4]:
eststo: csdid2 lnWhite_app SimpleATT lnpop* lncpop*, ivar(unitid) time(year) gvar(gvar) ///
        method(reg) agg(simple) notyet

Producing Long Gaps by default
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5 
..................................................    50
..................................................   100
..................................................   150
..................................................   200
............................
Difference-in-difference with Multiple Time Periods
Outcome model  : regression adjustment
Treatment model: none
------------------------------------------------------------------------------
             |               Robust
             | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
   SimpleATT |  -.1536081   .0608414    -2.52   0.012     -.272855   -.0343612
------------------------------------------------------------------------------
(est2 stored)


In [3]:
tab year if gvar==year & gvar<2014 & gvar!=2004 & gvar!=2003


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2006 |          2       16.67       16.67
       2009 |          1        8.33       25.00
       2010 |          3       25.00       50.00
       2011 |          1        8.33       58.33
       2012 |          1        8.33       66.67
       2013 |          4       33.33      100.00
------------+-----------------------------------
      Total |         12      100.00


In [5]:
eststo: sdid lnWhite_app unitid year SimpleATT ///
, vce(bootstrap) covariates(lnpop* lncpop*) seed(1234) reps(500)

Bootstrap replications (500). This may take some time.
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5
..................................................     50
..................................................     100
..................................................     150
..................................................     200
..................................................     250
..................................................     300
..................................................     350
..................................................     400
..................................................     450
..................................................     500


Synthetic Difference-in-Differences Estimator

-----------------------------------------------------------------------------
 lnWhite_app |     ATT     Std. Err.     t      P>|t|    [95% Conf. Interval]
-------------+---------------------------------------------------------------
   SimpleATT |  -0.162

In [8]:
eststo: sdid lnWhite_app unitid year SimpleATT if gvar<2014 & gvar!=2004 & gvar!=2003 ///
, vce(placebo) covariates(lnpop* lncpop*) seed(1234) reps(500)

Placebo replications (500). This may take some time.
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5
..................................................     50
..................................................     100
..................................................     150
..................................................     200
..................................................     250
..................................................     300
..................................................     350
..................................................     400
..................................................     450
..................................................     500


Synthetic Difference-in-Differences Estimator

-----------------------------------------------------------------------------
 lnWhite_app |     ATT     Std. Err.     t      P>|t|    [95% Conf. Interval]
-------------+---------------------------------------------------------------
   SimpleATT |  -0.22917

In [9]:
esttab , star(+ .1 * .05 ** .01 *** .001) ///
        stats(N_clust N, fmt(%9.0fc %9.0fc) label("schools" "N")) ///
        se(%9.3fc) b(%9.3fc) ///
        label mtitle("Panel DiD" "Panel SDiD Bootstrap" "Panel SDiD Placebo") ///
        nonotes modelwidth(10)

This front-end or document format cannot display HTML

In [8]:
qui esttab using ///
    tables/t2_did_applications.rtf, ///
    star(+ .1 * .05 ** .01 *** .001) ///
        stats(N_clust N, fmt(%9.0fc %9.0fc) label("schools" "N")) ///
        se(%9.3fc) b(%9.3fc) ///
        label mtitle("Panel DiD" "Panel SDiD Bootstrap" "Panel SDiD Placebo") ///
        nonotes modelwidth(10) replace


file tables/t2_did_applications.rtf already exists


r(602);
r(602);
